In [36]:
# Start Zookerp and Kafka Server (Each has to run in a seperate terminal)
#! zookeeper-server-start /usr/local/etc/kafka/zookeeper.properties
#! kafka-server-start /usr/local/etc/kafka/server.properties

[2021-02-01 16:16:14,330] INFO Reading configuration from: /usr/local/etc/kafka/zookeeper.properties (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2021-02-01 16:16:14,341] INFO clientPortAddress is 0.0.0.0:2181 (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2021-02-01 16:16:14,341] INFO secureClientPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2021-02-01 16:16:14,345] INFO autopurge.snapRetainCount set to 3 (org.apache.zookeeper.server.DatadirCleanupManager)
[2021-02-01 16:16:14,345] INFO autopurge.purgeInterval set to 0 (org.apache.zookeeper.server.DatadirCleanupManager)
[2021-02-01 16:16:14,345] INFO Purge task is not scheduled. (org.apache.zookeeper.server.DatadirCleanupManager)
[2021-02-01 16:16:14,345] WARN Either no config or no quorum defined in config, running  in standalone mode (org.apache.zookeeper.server.quorum.QuorumPeerMain)
[2021-02-01 16:16:14,350] INFO Log4j 1.2 jmx support found and enabled. (org.apache.zookeeper.jmx.ManagedUt

KeyboardInterrupt: 

In [48]:

from __future__ import unicode_literals
import json
from time import sleep

from bs4 import BeautifulSoup
from kafka import KafkaConsumer, KafkaProducer
import requests

# -*- coding: utf-8 -*-
calls_class = 'calls W(100%) Pos(r) Bd(0) Pt(0) list-options'
puts_class = 'puts W(100%) Pos(r) list-options'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
    'Pragma': 'no-cache'
    }

        
def get_trades(class_,
               base_url = 'https://finance.yahoo.com/quote/TSLA/options?straddle=false'):
    url = base_url
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find_all('table', class_ = class_)[0] # Grab the first table
    except Exception as ex:
        print('Exception in get_trades')
        print(str(ex))
    finally:
        return table

def publish_message(producer_instance, topic_name, key, value):
    try:
        #print(key)
        #print(value)
        key_bytes =  bytes(key, encoding='utf-8')   #.encode('utf-8')
        value_bytes = bytes(value, encoding='utf-8') #encode('utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

def connect_kafka_producer(server_address = ['localhost:9092']):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers = server_address, api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer



## Scrape Calls

In [47]:

calls = get_trades(class_ = calls_class)

kafka_producer = connect_kafka_producer()
publish_message(kafka_producer, 'TSLA_calls', 'raw', str(calls))
kafka_producer.close()

Accessing list
Message published successfully.


## Scrape Puts

In [50]:
puts = get_trades(class_ = puts_class)

kafka_producer = connect_kafka_producer()
publish_message(kafka_producer, 'TSLA_puts', 'raw', str(puts))
kafka_producer.close()

Accessing list
Message published successfully.


## Production Gist

This page is refreshed continuously, so the best way to track is to scrape the data continuously. We can deduplicate once the data data is in Spark (I think). 

In [51]:
ix = 0

while True:
    print('Scraping...')
    if ix == 10:
        print('Exiting')
        break
    calls = get_trades(class_ = calls_class)
    kafka_producer = connect_kafka_producer()
    publish_message(kafka_producer, 'TSLA_calls', 'raw', str(calls))
    kafka_producer.close()

    puts = get_trades(class_ = puts_class)
    kafka_producer = connect_kafka_producer()
    publish_message(kafka_producer, 'TSLA_puts', 'raw', str(puts))
    kafka_producer.close()
    ix += 1
    sleep(10)
    

Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published successfully.
Accessing list
Message published successfully.
Scraping...
Accessing list
Message published s